## Making room and equipment reservations via LibCal API

#### 1. Get Token

In [1]:
import requests
import configparser

config = configparser.ConfigParser()

config.read('config.ini')
url = config['DEFAULT']['url']
client_id = config['DEFAULT']['client_id']
client_secret = config['DEFAULT']['client_secret']
grant_type = config['DEFAULT']['grant_type']


def get_access_token():
    response = requests.post(
        url,
        data={"grant_type": grant_type},
        auth=(client_id, client_secret),
    )
    return response.json()

t = get_access_token()

# To check if getting token successed
print(t)


{'access_token': '6e82deb2c3a5f8358537c3614e1f0dc554157366', 'expires_in': 3600, 'token_type': 'Bearer', 'scope': 'rm_r sp_r'}


#### 2. Read Category Rooms Reservations

In [2]:
# king_swap = config['CATEGORIES']['swap_access']
king_swap = config['CATEGORIES']['key_only']
booking_url = config['DEFAULT']['booking_url']

header = {'Authorization': f'Bearer {t["access_token"]}'}

response = requests.get(booking_url.format(king_swap), headers=header)

# To check if reading rooms availability is successful
bookings = response.json()

print(bookings)

[{'bookId': 'cs_xGmV2gUX', 'id': 36548792, 'eid': 31095, 'cid': 6840, 'lid': 2047, 'fromDate': '2022-08-01T09:00:00-04:00', 'toDate': '2022-08-01T15:00:00-04:00', 'created': '2022-08-01T09:05:07-04:00', 'firstName': 'Parks', 'lastName': 'Marilla', 'email': 'king@king.com', 'account': 'king@king.com', 'status': 'Confirmed', 'location_name': 'King Library', 'category_name': 'King Study Rooms (Key Access Only)', 'item_name': 'King 128'}]


#### 3. Check if user ID is found in the list of reservations. Return booking index if found.

In [5]:
booking_index = -1

def hasUserID(userID):
    global booking_index
    for idx, booking in enumerate(bookings):
        status = booking['status']
        bid = booking['bookId']
        uid = booking['email'].split("@",1)[0]

        if uid == userID:
            booking_index = idx
            print("Booking id {} is {}".format(bid, status))
        else:
            print("Not found.")

print(hasUserID('king'))
print(booking_index)


Booking id cs_V7mal2uA is Confirmed
Booking id cs_mXwR9ZFb is Confirmed
None
1


#### 4. Check Time Difference

In [4]:
from datetime import datetime
import pytz
import dateutil.parser

booking = list(bookings)[booking_index]
status = booking['status']
bid = booking['bookId']
uid = booking['email'][:-12]
loc = booking['eid']
item_name = booking['item_name']
fromDate = dateutil.parser.isoparse(booking['fromDate'])
toDate = dateutil.parser.isoparse(booking['toDate'])
timeperiod = (toDate - fromDate).seconds/3600
print("Your have booked {} hours".format(timeperiod))

EST = pytz.timezone('US/Eastern')
dateTime_now = datetime.now(EST).isoformat('T',"seconds")
parsed_dtnow = dateutil.parser.isoparse(dateTime_now)
# parsed_dtnow = dateutil.parser.isoparse('2022-06-23 18:52:13-04:00')

def checkin_time_relative():
    # student checkin before the starting time
    if parsed_dtnow < fromDate:
        return 1
    # student checkin during the reservation period
    elif parsed_dtnow < toDate and parsed_dtnow > fromDate:
        return 2
    # student checkin after the ending time 
    elif parsed_dtnow > toDate:
        return 3
    # error
    else:
        return -1

def time_difference(sometime):
    timediffer = lambda sometime: sometime - parsed_dtnow if sometime > parsed_dtnow else parsed_dtnow - sometime

    tf_hours = timediffer(sometime).seconds / 3600
    tf_minutes = timediffer(sometime).seconds / 60

    if tf_hours > 1:
        return str(int(tf_hours)) + " hours " + str(int(tf_minutes) - int(tf_hours) * 60) + " mintues"
    else:
        return str(int(tf_minutes)) +  " minutes"

notice_str = "Your reservation of " + item_name + " {} {}."

if checkin_time_relative() == 1:
    print(notice_str.format("will start in", time_difference(fromDate)))
if checkin_time_relative() == 2:
    print(notice_str.format("has started for", time_difference(fromDate)))
if checkin_time_relative() == 3:
    print(notice_str.format("has ended for", time_difference(toDate)))
if checkin_time_relative() == -1:
    print("This is an error. Your reservation starts at {}, ends at {}".format(fromDate, toDate))

Your have booked 2.0 hours
Your reservation of King 018 will start in 2 hours 22 mintues.


# Archived: 3. Book a Room

In [9]:
# E.g. Book "Chat-Bot-Testing" building, Room1

reserve_url = config['DEFAULT']['resv_url']

# Put customized reservation information below:
start_time = "2022-04-30T22:00:00-05:00"
room_id = config['ROOMS']['Room1']
customer_fname = "FNAME"
customer_lname = "LNAME"
email_address = "example@abcabc.com"
end_time = "2022-04-30T22:15:00-05:00"


payload = {
    "start": start_time,
    "fname": customer_fname,
    "lname": customer_lname,
    "email": email_address,
    # "adminbooking": 1,
    "bookings": [
        {
            "id": room_id,
            "to": end_time
        },
    ]
}

response = requests.post(reserve_url, headers=header, json=payload)

# Return the confirmation: booking_id, indicates that the reservation is successful.
response.json()


{'booking_id': 'cs_g4Al7mFg', 'cost': '0'}

#### 4. Delete an Existing Room Reservation

In [10]:
cancel_url = config['DEFAULT']['cancel_url']
booking_id = "cs_g4Al7mFg"

cancel_response = requests.post(f'{cancel_url}/{booking_id}', headers=header)

# Return "True" as cancel successed
cancel_response.json()

[{'booking_id': 'cs_g4Al7mFg', 'cancelled': True}]

In [ ]:
!pip install requests